In [1]:
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

from torch_geometric.loader import DataLoader
from CustomDataset import CustomDataset

%config InlineBackend.figure_format = 'png2x'
plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'  # 'stix' is similar to Times New Roman
# If you want to use LaTeX to render all text (for more consistency)
# plt.rcParams['text.usetex'] = True

plt.rcParams['axes.linewidth'] = 0.5    # Axis line width
plt.rcParams['lines.linewidth'] = 1    # Line width
plt.rcParams['xtick.major.width'] = 0.5  # Set x major tick width to 2 points
plt.rcParams['ytick.major.width'] = 0.5
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['lines.markersize'] = 4
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## 1. Overall shedding

The event of load shedding at time $t$ is defined as:
$$\Psi_t := \Phi_t > 0$$

In [2]:
## Read shedding in test data
shedding_true = []
shedding_pred = []

test_size = 200

for i in range(test_size):
    # File path
    true_path = f'../../res/IEEE_Case1354/model_evaluation/agg_Shedding/true/true_{i+1}.csv'
    pred_path = f'../../res/IEEE_Case1354/model_evaluation/agg_Shedding/pred/pred_{i+1}.csv'

    # Read data
    try:
        temp1 = pd.read_csv(true_path, header=None, index_col=None)
        temp2 = pd.read_csv(pred_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    # Concatenate all data
    shedding_true.append(temp1.values.copy())
    shedding_pred.append(temp2.values.copy())

# Convert to numpy array and set precision
shedding_true = np.abs(np.array(shedding_true).astype(np.int32))
shedding_pred = np.abs(np.array(shedding_pred).astype(np.int32))

############################################################################
# Set prediction shreshold as 5 (prediction < 5 is considered as no shedding)
############################################################################
shedding_pred[shedding_pred < 5] = 0

In [3]:
# Calculate probability
prob_true = np.mean((shedding_true>0), axis=0)
prob_pred = np.mean((shedding_pred>0), axis=0)

In [ ]:
columns = [f'T={i+1}' for i in range(12)]

pd.DataFrame(prob_true, columns=columns)

In [ ]:
pd.DataFrame(prob_pred, columns=columns)

## 2. Shedding from non-reserve constraints

In [5]:
## Read shedding in test data
shedding_zero_reserve_true = []
shedding_zero_reserve_pred = []

test_size = 200

for i in range(test_size):
    # File path
    true_path = f'../../res/IEEE_Case1354/model_evaluation/agg_Shedding_zero_reserve/true/true_{i+1}.csv'
    pred_path = f'../../res/IEEE_Case1354/model_evaluation/agg_Shedding_zero_reserve/pred/pred_{i+1}.csv'

    # Read data
    try:
        temp1 = pd.read_csv(true_path, header=None, index_col=None)
        temp2 = pd.read_csv(pred_path, header=None, index_col=None)
    except FileNotFoundError:
        print(f'The file sample_{i+1} is not found')
        continue

    # Concatenate all data
    shedding_zero_reserve_true.append(temp1.values.copy())
    shedding_zero_reserve_pred.append(temp2.values.copy())

# Convert to numpy array and set precision
shedding_zero_reserve_true = np.abs(np.array(shedding_zero_reserve_true).astype(np.int32))
shedding_zero_reserve_pred = np.abs(np.array(shedding_zero_reserve_pred).astype(np.int32))

############################################################################
# Set prediction shreshold as 5 (prediction < 5 is considered as no shedding)
############################################################################
shedding_zero_reserve_pred[shedding_zero_reserve_pred < 5] = 0

In [6]:
# Calculate probability
prob_zero_reserve_true = np.mean((shedding_zero_reserve_true>0), axis=0)
prob_zero_reserve_pred = np.mean((shedding_zero_reserve_pred>0), axis=0)

## 3. Shedding from reserve constraints

In [7]:
# Calculate probability
prob_reserve_true = prob_true - prob_zero_reserve_true
prob_reserve_pred = prob_pred - prob_zero_reserve_pred

# Get absolute values for probability
prob_reserve_true = np.abs(prob_reserve_true)
prob_reserve_pred = np.abs(prob_reserve_pred)

In [ ]:
fig = plt.figure(figsize=(12, 1.75))

x = np.arange(1, 13)

ax = fig.add_subplot(141)
ax.plot(x, prob_zero_reserve_true[0], linestyle='dashed', color='red', marker='s', label=None)
ax.plot(x, prob_zero_reserve_pred[0], linestyle='dashed', color='blue', marker='s', label=None)
ax.plot(x, prob_reserve_true[0], linestyle='dotted', color='red', marker='s', label=None)
ax.plot(x, prob_reserve_pred[0], linestyle='dotted', color='blue', marker='s', label=None)
ax.scatter([], [], color='red', marker='s', label='MILP')
ax.scatter([], [], color='blue', marker='s', label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((0, 1))
ax.set_ylabel('$\mathcal{P}_s(t)$')
ax.legend(loc='upper left', ncols=2, frameon=False)

ax = fig.add_subplot(142)
ax.plot(x, prob_zero_reserve_true[1], linestyle='dashed', color='red', marker='s', label=None)
ax.plot(x, prob_zero_reserve_pred[1], linestyle='dashed', color='blue', marker='s', label=None)
ax.plot(x, prob_reserve_true[1], linestyle='dotted', color='red', marker='s', label=None)
ax.plot(x, prob_reserve_pred[1], linestyle='dotted', color='blue', marker='s', label=None)
ax.plot([], [], linestyle='dotted', color='red', label='Reserve related')
ax.plot([], [], linestyle='dashed', color='red', label='Non-reserve related')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((0, 1))
ax.set_ylabel('$\mathcal{P}_s(t)$')
ax.legend(loc='upper left', frameon=False)

ax = fig.add_subplot(143)
ax.plot(x, prob_zero_reserve_true[2], linestyle='dashed', color='red', marker='s', label='MILP')
ax.plot(x, prob_zero_reserve_pred[2], linestyle='dashed', color='blue', marker='s', label='GNN')
ax.plot(x, prob_reserve_true[2], linestyle='dotted', color='red', marker='s', label='MILP')
ax.plot(x, prob_reserve_pred[2], linestyle='dotted', color='blue', marker='s', label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((0, 1))
ax.set_ylabel('$\mathcal{P}_s(t)$')
# ax.legend(loc='upper right', ncols=2, frameon=False)

ax = fig.add_subplot(144)
ax.plot(x, prob_zero_reserve_true[3], linestyle='dashed', color='red', marker='s', label='MILP')
ax.plot(x, prob_zero_reserve_pred[3], linestyle='dashed', color='blue', marker='s', label='GNN')
ax.plot(x, prob_reserve_true[3], linestyle='dotted', color='red', marker='s', label='MILP')
ax.plot(x, prob_reserve_pred[3], color='blue', marker='s', label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((0, 1))
ax.set_ylabel('$\mathcal{P}_s(t)$')
# ax.legend(loc='upper right', ncols=2, frameon=False)

plt.tight_layout()
plt.show()
plt.close('all')

## Conditional probability
Conditional probability is defined as: 
$$\Delta T = 2 h$$
$$\mathcal{P}_s^+(t) = p\left(\Psi_{t+1} \cup \Psi_{t+2} | \Psi_t\right)$$

In [9]:
# Number of zones
num_zones= prob_true.shape[0]
# Number of time steps
T = prob_true.shape[-1]
# Time increament for conditional probability
delta_T = 2

cond_prob_true = np.zeros(prob_true.shape)
cond_prob_pred = np.zeros(prob_pred.shape)

# Calculate conditional probability ground truth
for i in range(num_zones):
    for j in range(T-delta_T):
        idx = shedding_true[:, i, j] > 0
        num = (shedding_true[idx, i, j+1:j+1+delta_T] > 0).sum(axis=1)
        cond_prob_true[i, j] = (num >= 1).sum() / idx.sum()

# Calculate conditional probability prediction
for i in range(num_zones):
    for j in range(T-delta_T):
        idx = shedding_pred[:, i, j] > 0
        num = (shedding_pred[idx, i, j+1:j+1+delta_T] > 0).sum(axis=1)
        cond_prob_pred[i, j] = (num >= 1).sum() / idx.sum()

In [ ]:
fig = plt.figure(figsize=(12, 1.75))

x = np.arange(1, 11)

ax = fig.add_subplot(141)
ax.plot(x, cond_prob_true[0][:-delta_T], color='red', marker='s', label='Reserve related')
ax.plot(x, cond_prob_pred[0][:-delta_T], color='blue', marker='s', label=None)
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((-0.1, 1.1))
ax.set_ylabel('$\mathcal{P}^+_s(t)$')
# ax.legend(loc='upper left', frameon=False)

ax = fig.add_subplot(142)
ax.plot(x, cond_prob_true[1][:-delta_T], color='red', marker='s', label='MILP')
ax.plot(x, cond_prob_pred[1][:-delta_T], color='blue', marker='s', label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((-0.1, 1.1))
ax.set_ylabel('$\mathcal{P}^+_s(t)$')
# ax.legend(loc='upper right', ncols=2, frameon=False)

ax = fig.add_subplot(143)
ax.plot(x, cond_prob_true[2][:-delta_T], color='red', marker='s', label='MILP')
ax.plot(x, cond_prob_pred[2][:-delta_T], color='blue', marker='s', label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((-0.1, 1.1))
ax.set_ylabel('$\mathcal{P}^+_s(t)$')
# ax.legend(loc='upper right', ncols=2, frameon=False)

ax = fig.add_subplot(144)
ax.plot(x, cond_prob_true[3][:-delta_T], color='red', marker='s', label='MILP')
ax.plot(x, cond_prob_true[3][:-delta_T], color='blue', marker='s', label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((-0.1, 1.1))
ax.set_ylabel('$\mathcal{P}^+_s(t)$')
# ax.legend(loc='upper right', ncols=2, frameon=False)

plt.tight_layout()
plt.show()
plt.close('all')

## Risk

Risk is defined as the consequence (monetary cost) of a failure event, and is calculated as the product of probability of that failure event and the predefined economic loss. However, consequence depends on the degree of failure, not simply whether that failure event happens or not. To encompass the degree of failure, risk is therefore calculated as the expectation of risk associated with MC samples.

Risk associated with individual sample at $t$ is: $$R_s\left( t \right) = \int_0^{\psi(t)} C_s(\psi) d\psi,$$

risk at time $t$ is obtained as: $$\mathcal{R}_s(t) = \mathbb{E}\left[ R_s(t) \right],$$

risk form future $\Delta T$ time steps exerted at $t$ is calculated as: $$\mathcal{R}_s^+(t) = \sum_{\Delta t = 1}^{\Delta T} \mathbb{E}\left[ R_s(t + \Delta t) \right]$$

The overall risk is: $$\Re_s(t) = \mathcal{R}_s(t) + \mathcal{R}_s^+(t)$$ $$= \mathbb{E}\left[ R_s(t) \right] + \sum_{\Delta t = 1}^{\Delta T} \mathbb{E}\left[ R_s(t + \Delta t) \right]$$ $$= \sum_{\Delta t = 0}^{\Delta T} \mathbb{E}\left[ R_s(t + \Delta t) \right]$$

In [11]:
# Define cost function
Cs = 10 # Cost of shedding, $/MW

## Risk associated with shedding at time t
cost_true = shedding_true * Cs
cost_pred = shedding_pred * Cs
risk_true = np.mean(cost_true, axis=0)
risk_pred = np.mean(cost_pred, axis=0)

## Risk associated with shedding between [t+1, t+delta_T]
cond_risk_true = np.zeros(risk_true.shape)
cond_risk_pred = np.zeros(risk_pred.shape)

## Calculate conditional risk
# Ground truth
for i in range(num_zones):
    for j in range(T-delta_T):
        idx = shedding_true[:, i, j] > 0
        cond_cost_true = shedding_true[:, i, j+1:j+1+delta_T] * Cs
        cond_risk_true[i, j] = cond_cost_true.mean(axis=0).sum()
## Prediction
for i in range(num_zones):
    for j in range(T-delta_T):
        idx = shedding_pred[:, i, j] > 0
        cond_cost_pred = shedding_pred[:, i, j+1:j+1+delta_T] * Cs
        cond_risk_pred[i, j] = cond_cost_pred.mean(axis=0).sum()

## Calculate overall risk
overall_risk_true = risk_true + cond_risk_true
overall_risk_pred = risk_pred + cond_risk_pred

In [ ]:
fig = plt.figure(figsize=(12, 1.75))

x = np.arange(1, 13)

ax = fig.add_subplot(141)
ax.plot(x, risk_true[0]/1000, color='red', marker='x', markersize=6, label='MILP')
ax.plot(x, risk_pred[0]/1000, color='blue', marker='x', markersize=6, label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ [h]')
ax.set_ylim((0, 4))
ax.set_ylabel('$\mathcal{R}(t)$ [$\\times 10^3$]')
ax.legend(loc='lower left', ncols=2, frameon=False)

ax = fig.add_subplot(142)
ax.plot(x, risk_true[1]/1000, color='red', marker='x', markersize=6, label='MILP')
ax.plot(x, risk_pred[1]/1000, color='blue', marker='x', markersize=6, label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ [h]')
ax.set_ylim((0, 1))
ax.set_ylabel('$\mathcal{R}(t)$ [$\\times 10^3$]')
# ax.legend(loc='upper right', ncols=2, frameon=False)

ax = fig.add_subplot(143)
ax.plot(x, risk_true[2]/1000, color='red', marker='x', markersize=6, label='MILP')
ax.plot(x, risk_pred[2]/1000, color='blue', marker='x', markersize=6, label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ [h]')
ax.set_ylim((0, 4))
ax.set_ylabel('$\mathcal{R}(t)$ [$\\times 10^3$]')
# ax.legend(loc='upper right', ncols=2, frameon=False)

ax = fig.add_subplot(144)
ax.plot(x, risk_true[3]/1000, color='red',  marker='x', markersize=6, label='MILP')
ax.plot(x, risk_pred[3]/1000, color='blue', marker='x', markersize=6, label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ [h]')
ax.set_ylim((0, 2))
ax.set_ylabel('$\mathcal{R}(t)$ [$\\times 10^3$]')
# ax.legend(loc='upper right', ncols=2, frameon=False)

plt.tight_layout()
plt.show()
fig.savefig(f'../../res/IEEE_Case1354/reliability_risk/shedding_risk.pdf', bbox_inches='tight')
plt.close('all')

In [ ]:
fig = plt.figure(figsize=(12, 1.75))

x = np.arange(1, 11)

ax = fig.add_subplot(141)
ax.plot(x, overall_risk_true[0][:-delta_T]/1000, color='red', marker='x', markersize=6, label='Reserve related')
ax.plot(x, overall_risk_pred[0][:-delta_T]/1000, color='blue', marker='x', markersize=6, label=None)
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((0, 12))
ax.set_ylabel('$\Re_s(t)$ $(\$10^3)$')
# ax.legend(loc='upper left', frameon=False)

ax = fig.add_subplot(142)
ax.plot(x, overall_risk_true[1][:-delta_T]/1000, color='red', marker='x', markersize=6, label='MILP')
ax.plot(x, overall_risk_pred[1][:-delta_T]/1000, color='blue', marker='x', markersize=6, label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((0, 12))
ax.set_ylabel('$\Re_s(t)$ $(\$10^3)$')
# ax.legend(loc='upper right', ncols=2, frameon=False)

ax = fig.add_subplot(143)
ax.plot(x, overall_risk_true[2][:-delta_T]/1000, color='red', marker='x', markersize=6, label='MILP')
ax.plot(x, overall_risk_pred[2][:-delta_T]/1000, color='blue', marker='x', markersize=6, label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((0, 12))
ax.set_ylabel('$\Re_s(t)$ $(\$10^3)$')
# ax.legend(loc='upper right', ncols=2, frameon=False)

ax = fig.add_subplot(144)
ax.plot(x, overall_risk_true[3][:-delta_T]/1000, color='red', marker='x', markersize=6,  label='MILP')
ax.plot(x, overall_risk_pred[3][:-delta_T]/1000, color='blue', marker='x', markersize=6, label='GNN')
ax.set_xlim((0, 13))
ax.set_xticks([2, 4, 6, 8, 10, 12])
ax.set_xlabel('Time step $t$ (h)')
ax.set_ylim((0, 12))
ax.set_ylabel('$\Re_s(t)$ $(\$10^3)$')
# ax.legend(loc='upper right', ncols=2, frameon=False)

plt.tight_layout()
plt.show()
plt.close('all')